# Youbike 2.0 Data Preprocessing (2)

1. get the one stop from each section 
    我自己是肉眼看挑比較離台大不要太遠、比較有代表性、比較大的站點，沒特別想法的話就選站點1。    
    （https://drive.google.com/drive/folders/1oTzk_Mel65__q8qR9tv4TFEtk5vrnnAB）
    
2. select from the distance matrix, the matrix is saved @ mini_dmat.txt 
    (with depot node)
   , and mini_dmat_withou_depot.txt is without depot node. 
   , select and reorder the dataframe to mini_Ubike站點供給.csv
   
3. output testcases **without n_vehickle and capacity** into mini_testcase_早九 and mini_testcase_晚五

In [48]:
stoppath = './data/Ubike站點供給_v3.csv'
mappingpath = './data/idx2stop.pkl'
import pandas as pd
import joblib
import numpy as np

In [41]:

df = pd.read_csv(stoppath)
idx2stop = joblib.load(mappingpath)

In [17]:
# add section-idx to this mapping
stop2idx = {v:k for k, v in idx2stop.items()}
for i in range(len(df)):
    row = df.iloc[i]
    
    stopname = row['站位名稱']
    sectionidx = row['站位編號']
    matidx = stop2idx[stopname]
    stop2idx[stopname] = [matidx, sectionidx]
# stop2idx


In [23]:
# stop2idx

In [19]:
keeps = ['1-2', '2-1', '3-4',
         '4-1', '5-3', '6-1', 
         '7-4', '8-1', '9-1', 
         '10-1', '11-1', '12-1', 
         '13-1', '14-1', '15-1', '16-4']

In [20]:
mini_stop2idx = {}
for k, v in stop2idx.items():
    if v[1] in keeps:
        mini_stop2idx[k] = v
# joblib.dump(stop2idx, './data/mini_stop2idx.pkl')

In [24]:
mini_stop2idx

{'新生南路三段82號前': [3, '16-4'],
 '捷運公館站(2號出口)': [11, '1-2'],
 '臺大法人語言訓練中心前': [15, '10-1'],
 '辛亥新生路口東南側': [17, '13-1'],
 '臺大天文數學館南側': [29, '11-1'],
 '臺大舊體育館西側': [34, '12-1'],
 '臺大公館停車場西北側': [38, '3-4'],
 '臺大第二行政大樓南側': [39, '6-1'],
 '臺大明達館機車停車場': [40, '8-1'],
 '臺大第一活動中心西南側': [51, '7-4'],
 '臺大博理館西側': [52, '9-1'],
 '臺大一號館': [55, '5-3'],
 '臺大小小福西南側': [56, '4-1'],
 '捷運科技大樓站': [66, '14-1'],
 '臺大水源舍區B棟': [78, '2-1'],
 '捷運臺電大樓站(2號出口)': [81, '15-1']}

In [29]:
N_NODE = 16

In [43]:
def make_new_bi(df, 
                n_node = N_NODE):
    def get_bi(sup_pre, dem, pole_pre):
        init = sup_pre - dem 
        if init < 0 and abs(init) > pole_pre:
            return - pole_pre
        return init 
    for time in ['早九', '晚五']:
        df[f'{time}_bi'] = df.apply(lambda x: get_bi(x[f'{time}可借車數_pre'], x[f'{time}借車人數'], x[f'{time}空柱數量_pre']), axis = 1)
    keepindices = []
    for i in range(len(df)):
        row = df.iloc[i]
        stopname = row['站位名稱']
        sectionidx = row['站位編號']
        if sectionidx in keeps:
            keepindices.append(i)

    # df.drop(df.index[[1,3]])
    return keepindices

In [44]:
to_keep_rows = make_new_bi(df)
df = df.iloc[to_keep_rows]
# df.to_csv('./data/mini_Ubike站點供給.csv', index = False)

In [46]:
mini_stop2idx

In [92]:
dmatpath = './data/mtx_distance.csv'
dmat = pd.read_csv(dmatpath)
dmat = dmat.to_numpy()
print(dmat.shape)

(81, 81)


In [93]:
keepidx = []
for k, v in mini_stop2idx.items():
    keepidx.append(v[0])
keepidx = [i-1 for i in keepidx]
keepidx 
dmat = dmat[keepidx, :][:,keepidx]
print(dmat.shape)
newd = np.zeros((N_NODE+1, N_NODE +1))  
newd[1:, 1:] = dmat
np.savetxt('./data/mini_dmat_without_depot.txt', dmat, delimiter=',')   
np.savetxt('./data/mini_dmat.txt', newd, delimiter=',')

(16, 16)


In [68]:
keepidx

[2, 10, 14, 16, 28, 33, 37, 38, 39, 50, 51, 54, 55, 65, 77, 80]

Make 2 mini-testcases for 早九 and 晚五 respectively

In [78]:
s = sorted(mini_stop2idx.values(), key= lambda x:x[0])
ordered_secidx = [x[1] for x in s]
ordered_secidx

['16-4',
 '1-2',
 '10-1',
 '13-1',
 '11-1',
 '12-1',
 '3-4',
 '6-1',
 '8-1',
 '7-4',
 '9-1',
 '5-3',
 '4-1',
 '14-1',
 '2-1',
 '15-1']

In [87]:
# reorder dataframe for distance matrix 
df_ = df.set_index("站位編號", drop = True)
df_ = df_.reindex(ordered_secidx)
df_ = df_.reset_index(drop = False)

In [89]:
#df_

In [90]:
# reorder the dataframe to correpond to the distance matrix
df_.to_csv('./data/mini_Ubike站點供給.csv', index = False)

In [96]:
def gen_cases(data = df_, 
              dmatrixpath = './data/mini_dmat.txt'):
    dmat = np.loadtxt(dmatrixpath, delimiter=',')
    for time in ['早九', '晚五']:
        
        dire = './data/mini-testcases'
        file_prefix = f'{dire}/mini_testcase'
        b = data[f'{time}_bi'].tolist()
        b.insert(0, -sum(b))
        testcase = {'n_node':N_NODE, 'b':b, 'd':dmat} 
        joblib.dump(testcase, f'{file_prefix}_{time}')
gen_cases()

### test loading

In [99]:
case = joblib.load('./data/mini-testcases/mini_testcase_早九')
print(case['d'].shape)
len(case['b'])


(17, 17)


17